## Array Computation with Dynamic Compute - GeoContexts
In previous notebooks we have explored the interactive visualization component of `Dynamic Compute`. Here we will dive into how and when we can extract the underlying pixel data associated with our `Mosaic` and `ImageStack` objects as `numpy ndarrays`.

In [ ]:
import descarteslabs as dl
import descarteslabs.dynamic_compute as dc

## Intro to GeoContexts 
Before we pull down any pixel data we must first define an area of interst (AOI) and several raster metadata parameters by which we organize our dataset. At Descartes Labs this is where the `GeoContext` comes into play--we have several operators to create and utilize `GeoContext` objects, which we will introduce in this notebook.

First, we'll define an interactive `map` as we have in other examples and an associated `sentinel-2:l2a` `Mosaic` and `ImageStack`. Here we will be returning to [Kossuth County, Iowa's Highest Corn Producing County](https://www.nass.usda.gov/Statistics_by_State/Iowa/Publications/County_Estimates/2021/IA-CtyEst-Corn-02-21.pdf).

In [ ]:
m = dc.map
m.center = 43.197541, -94.221831
m.zoom = 13

In [ ]:
s2_mosaic = dc.Mosaic.from_product_bands(
    "esa:sentinel-2:l2a:v1",
    "nir red green",
    start_datetime="2022-06-01",
    end_datetime="2022-09-01",
)

s2_stack = dc.ImageStack.from_product_bands(
    "esa:sentinel-2:l2a:v1",
    "nir red green",
    start_datetime="2022-06-01",
    end_datetime="2022-09-01",
).filter(lambda x: x.cloud_fraction < 0.1)

_ = s2_mosaic.visualize("FCC", m, scales=[[0, 1], [0, 1], [0, 1]])
_ = s2_stack.median(axis="images").visualize(
    "FCC-Cloudfree", m, scales=[[0, 1], [0, 1], [0, 1]]
)

In [ ]:
m

### Interactive Map GeoContexts

The simplest `GeoContext` object to retrieve is that of your interactive `map`. We can simply call `map.geocontext()` to retrieve the current viewport as a `GeoContext`:

In [ ]:
geocontext = m.geocontext()
type(geocontext)

Note this `GeoContext`'s attributes, which may vary depending on the provenance of the particular object:
* `crs`: EPSG code
* `bounds`: bounding rectangle of the viewport
* `bounds_crs`: EPSG code
* `shape`: shape of the resulting array

In [ ]:
geocontext

### Putting the _Compute_ in Dynamic Compute
Once we have _either_ a `Mosaic` or `ImageStack` _and_ a properly defined `GeoContext` we can now retrieve our pixel data as a `numpy array`.
To do this we simply choose which bands we want to pull down and call `.compute(geocontext)`. Note the resulting data type is a `DotDict`:

In [ ]:
s2_mosaic_data = s2_mosaic.pick_bands("nir red green").compute(geocontext)
type(s2_mosaic_data)

At this point we have _already retrieved the array we want_. We can access that data by calling `.ndarray` on our results. Note that if we have `shape` defined in our `GeoContext` that our resulting array's will be `(nbands, nx, ny)`:

In [ ]:
s2_mosaic_data.ndarray.shape

And finally we can use `dl.utils.display` to plot our dataset as an RGB:

In [ ]:
dl.utils.display(s2_mosaic_data.ndarray)

We also have `properties` returned to us in our results dictionary, which we will return to later on:

In [ ]:
s2_mosaic_data.properties

We note there is some pesky cloud and cloud shadows present in our scene, but also recall that we have the _temporal dimension_ exposed to us through our `ImageStack`. We can also pull down that _entire stack of data_ through calling `ImageStack.compute(geocontext)`. Note the resulting array's shape here will be `(nsamples, nbands, nx, ny)`:

In [ ]:
s2_stack_data = s2_stack.pick_bands("nir red green").compute(geocontext)
s2_stack_data.ndarray.shape

Here we will return to our `properties`, where the metadata is much more useful than in our previous `Mosaic` example. We can retrieve each Image's date through inline list comprehension:

In [ ]:
props = s2_stack_data.properties
dates = [p["acquired"].strftime("%Y-%m-%d %HH-%MM-%SS") for p in props]
dates

In [ ]:
ids = [p["id"] for p in props]
ids

In [ ]:
titles = [f"{ids[i]} \n {dates[i]}" for i in range(len(dates))]

### Pulling it All Together - ImageStacks
Note here that each Image we retrieved in this stack _may not completely cover our input AOI_, that is because we have found an area _on the boundary between Sentinel-2 Imagery collections_. In the below plot we label each image with it's associated collectoin time as well as it's unique Image ID: (This may be a bit too much??)

Note here that not each indvidual image covers the entire input AOI, this is because we are plotting individual Sentinel-2 strips!

In [ ]:
dl.utils.display(*s2_stack_data.ndarray, title=titles)